In [2]:
n_filters = 64
dilation_depth = 8
activation = 'softmax'
scale_ratio = 1
kernel_size = 2
pool_size_1 = 4
pool_size_2 = 8
traget=5
batch_size = 32
input_shape=(16000,1)

In [3]:
import glob
import librosa
import numpy as np

data1 = []
labels1 = []

CAS = glob.glob('/kaggle/input/primaryicbhi2017/mixed_5s/Asthma/*.wav')
CASandDAS = glob.glob('/kaggle/input/primaryicbhi2017/mixed_5s/Asthma/*.wav')
DAS = glob.glob('/kaggle/input/primaryicbhi2017/mixed_5s/COPD/*.wav')
Normal = glob.glob('/kaggle/input/primaryicbhi2017/mixed_5s/Healthy/*.wav')
Poor_Quality = glob.glob('/kaggle/input/primaryicbhi2017/mixed_5s/Pneumonia/*.wav')


SAMPLE_RATE=4000

nsample=16000
for file_path in CAS:   
    #print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    signal=np.array(signal[0:nsample])
#     print(signal.shape)
    data1.append(signal)
    labels1.append(0)

for file_path in CASandDAS:   
   # print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    signal=np.array(signal[0:nsample])
#     print(signal.shape)
    data1.append(signal)
    labels1.append(1)
for file_path in DAS:   
    #print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

    signal=np.array(signal[0:nsample])
#     print(signal.shape)
    data1.append(signal)
    labels1.append(2)

for file_path in Normal:   
   # print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)

    
    signal=np.array(signal[0:nsample])
#     print(signal.shape)
    data1.append(signal)
    labels1.append(3)
for file_path in Poor_Quality:   
    #print(file_path)
    signal, sample_rate = librosa.load(file_path, sr=SAMPLE_RATE)
    
    signal=np.array(signal[0:nsample])
    if signal.shape==(nsample,):
#     print(signal.shape)
        data1.append(signal)
        labels1.append(4)
    else:
        continue
X_train= np.array(data1)
y_train= np.array(labels1)


In [4]:
np.unique(y_train,return_counts=True)

(array([0, 1, 2, 3, 4]), array([431, 431, 805, 660, 260]))

Make the residual block of the WaveNet

In [5]:
from keras.layers import Conv1D, Input, Activation, AveragePooling1D, Add, Multiply, GlobalAveragePooling1D
from keras.models import Model
def residual_block(x, i):
    tanh_out = Conv1D(n_filters, 
                      kernel_size, 
                      dilation_rate = kernel_size**i, 
                      padding='causal',
                      activation='tanh'
                      )(x)
    sigm_out = Conv1D(n_filters, 
                      kernel_size, 
                      dilation_rate = kernel_size**i, 
                      padding='causal', 
                      activation='sigmoid'
                      )(x)
    z = Multiply()([tanh_out, sigm_out])
    skip = Conv1D(n_filters, 1)(z)
    res = Add()([skip, x])
    return res, skip
x = Input(shape=input_shape)
skip_connections = []
out = Conv1D(n_filters, 2, dilation_rate=1, padding='causal')(x)
for i in range(1, dilation_depth + 1):
    out, skip = residual_block(out,i)
    skip_connections.append(skip)
out = Add()(skip_connections)
out = Activation('relu')(out)
out = Conv1D(traget,1, padding='same', activation='relu')(out)
out = Conv1D(traget,1 , padding='same', activation='relu')(out)
out = GlobalAveragePooling1D()(out)
out = Activation(activation)(out)

model = Model(x, out)  
model.compile(optimizer='adam', 
                          loss='sparse_categorical_crossentropy', 
                          metrics=['accuracy'])
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 16000, 1)]   0           []                               
                                                                                                  
 conv1d (Conv1D)                (None, 16000, 64)    192         ['input_1[0][0]']                
                                                                                                  
 conv1d_1 (Conv1D)              (None, 16000, 64)    8256        ['conv1d[0][0]']                 
                                                                                                  
 conv1d_2 (Conv1D)              (None, 16000, 64)    8256        ['conv1d[0][0]']                 
                                                                                              

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(y_train),
                                        y = y_train                                                   
                                    )
class_weights = dict(zip(np.unique(y_train), class_weights))
class_weights

checkpoint_path = 'best.h5'
checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

import numpy as np
# from focal_loss import SparseCategoricalFocalLoss
from sklearn.model_selection import train_test_split

# Train the model
batch_size = 16
epochs = 100

model.fit(X_train, y_train,callbacks=[checkpoint],validation_split=.2,epochs=epochs ,class_weight=class_weights, batch_size=batch_size)

Epoch 1/100
130/130 [==============================] - ETA: 0s - loss: 1.1144 - accuracy: 0.5268
Epoch 1: val_accuracy improved from -inf to 0.03668, saving model to best.h5
130/130 [==============================] - 25s 195ms/step - loss: 1.1144 - accuracy: 0.5268 - val_loss: 2.6308 - val_accuracy: 0.0367
Epoch 2/100
130/130 [==============================] - ETA: 0s - loss: 1.0929 - accuracy: 0.5317
Epoch 2: val_accuracy did not improve from 0.03668
130/130 [==============================] - 24s 186ms/step - loss: 1.0929 - accuracy: 0.5317 - val_loss: 2.7043 - val_accuracy: 0.0193
Epoch 3/100
130/130 [==============================] - ETA: 0s - loss: 1.0844 - accuracy: 0.5423
Epoch 3: val_accuracy improved from 0.03668 to 0.04633, saving model to best.h5
130/130 [==============================] - 24s 183ms/step - loss: 1.0844 - accuracy: 0.5423 - val_loss: 3.1211 - val_accuracy: 0.0463
Epoch 4/100
130/130 [==============================] - ETA: 0s - loss: 1.0755 - accuracy: 0.5423
Ep

In [ ]:
y_train